In [20]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool


os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

@tool
def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def divide(a: int, b: int) -> int:
    """Divide two numbers."""
    return a / b


tools = [add, subtract, multiply, divide]

model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant with tools."),
        ("human", "{input}"), 
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

agent = create_tool_calling_agent(
    tools=tools,
    llm=model, 
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

response = agent_executor.invoke(
    {
        "input": "What is 5 plus 3 and what is 10 minus 4?"
    }
)

print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 5, 'b': 3}`


8
Invoking: `subtract` with `{'a': 10, 'b': 4}`


6The result of 5 plus 3 is 8 and the result of 10 minus 4 is 6.

> Finished chain.
The result of 5 plus 3 is 8 and the result of 10 minus 4 is 6.


In [ ]:
response = agent_executor.invoke(
    {
        "input": "What is 5 multiplied by 6 and what is 20 divided by 4?"
    }
)

print(response["output"])

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant with tools."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"), 
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

agent = create_tool_calling_agent(
    tools=tools,
    llm=model, 
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

chat_history = []

while True:
    user_input = input("Enter your question (or 'exit' to quit): ")
    print(user_input)
    
    if user_input.lower() == "exit":
        break
    
    response = agent_executor.invoke(
        {
            "input": user_input, 
            "chat_history": chat_history
        }
    )

    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response["output"]))
    
    print(response["output"])